# Exploring World Nations Statistics - Population | GDP | Languages

<img src="https://www.worldatlas.com/r/w2560-q80/upload/f4/e0/49/shutterstock-1665254809.png" alt='World Nations' width="500"><br>

This study aims to __investigate statistics surrounding nations around the world__, in particular population, GDP and languages spoken. SQL will be used to query the World Nations [MariaDB Sample Database](https://www.mariadbtutorial.com/getting-started/mariadb-sample-database/) to select specific data, and reveal information around a set of questions outlined below.

Some supplementary graphical representation will be used along the way to help visualize the queried data.

It is worth noting that the World Nations relational database consists of statistics between the years of 1960 and 2018.

__Project Questions being Explored__

<p>1. What countries have the highest GDP per Capita?</p>
<p>2. How much growth have the most populous countries experienced over recent decades?</p>
<p>3. What are the most densely populated countries?</p>
<p>4. Which languages are most commonly spoken? What countries speak the most different languages?</p>
<p>5. Is there a most common month for a countries' national days?</p>

## 🌏 1. __Exploring GDP per Capita of Countries__

*What countries have the highest GDP per Capita?*

We will begin our investigation by exploring <code>population</code>, <code>gdp</code> and <code>gdp_per_capita</code> to identify the top 10 highest GDP per Capita countries and some of their attributes.

In [54]:
-- Count number of unique countries in the data
SELECT COUNT(DISTINCT name) AS number_of_countries
FROM countries;

,number_of_countries
0,239


In [55]:
-- Select country name, GDP, population, GDP per capita (calculate gdp/population), continent name, and continent population
SELECT 	
	c.name AS country_name,
    gdp,
    population,
    gdp / population AS gdp_per_capita,
    con.name AS continent_name,
    SUM(population) OVER(PARTITION BY continent_name) AS continent_pop
-- Join tables from database
FROM countries AS c
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
LEFT JOIN regions AS r
ON c.region_id = r.region_id
LEFT JOIN continents AS con
ON r.continent_id = con.continent_id
-- Filter for Year 2018 only (the latest year available in 'country_stats' table)
WHERE year = 2018
	AND population IS NOT NULL 
    AND gdp IS NOT NULL
-- Sort by GDP per capita descending
ORDER BY gdp_per_capita DESC
LIMIT 12;

,country_name,gdp,population,gdp_per_capita,continent_name,continent_pop
0,Luxembourg,69487922619,607728,114340.4987,Europe,717216719
1,Macao,54545184268,631636,86355.4077,Asia,4376908610
2,Switzerland,705501000000,8516543,82838.8937,Europe,717216719
3,Norway,434751000000,5314336,81807.2098,Europe,717216719
4,Ireland,382487000000,4853506,78806.3309,Europe,717216719
5,Iceland,25882218777,353574,73201.7025,Europe,717216719
6,Qatar,192009000000,2781677,69026.3463,Asia,4376908610
7,Singapore,364157000000,5638676,64582.0047,Asia,4376908610
8,United States,20494100000000,327167434,62641.0146,North America,569629826
9,Denmark,352058000000,5797446,60726.3957,Europe,717216719


In [56]:
-- Select country name, GDP, GDP ranking, population, population ranking, GDP per capita, GDP per capita ranking
SELECT 	
	c.name AS country_name,
    gdp,
    ROW_NUMBER() OVER (ORDER BY gdp DESC) AS gdp_ranking,
    population,
    ROW_NUMBER() OVER (ORDER BY population DESC) AS population_ranking,
    gdp / population AS gdp_per_capita,
    ROW_NUMBER() OVER (ORDER BY gdp_per_capita DESC) AS gdp_per_capita_ranking
-- Join tables from database
FROM countries AS c
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
LEFT JOIN regions AS r
ON c.region_id = r.region_id
LEFT JOIN continents AS con
ON r.continent_id = con.continent_id
-- Filter for Year 2018 only (the latest year available in 'country_stats' table)
WHERE year = 2018
	AND population IS NOT NULL 
    AND gdp IS NOT NULL
-- Sort by GDP per capita descending
ORDER BY gdp_per_capita DESC
LIMIT 12;

,country_name,gdp,gdp_ranking,population,population_ranking,gdp_per_capita,gdp_per_capita_ranking
0,Luxembourg,69487922619,69,607728,156,114340.4987,1
1,Macao,54545184268,79,631636,155,86355.4077,2
2,Switzerland,705501000000,20,8516543,92,82838.8937,3
3,Norway,434751000000,27,5314336,109,81807.2098,4
4,Ireland,382487000000,30,4853506,113,78806.3309,5
5,Iceland,25882218777,103,353574,163,73201.7025,6
6,Qatar,192009000000,51,2781677,132,69026.3463,7
7,Singapore,364157000000,33,5638676,106,64582.0047,8
8,United States,20494100000000,1,327167434,3,62641.0146,9
9,Denmark,352058000000,36,5797446,105,60726.3957,10


### __Insights on GDP per Capita of countries__

- Amongst the top 10 GDP per Capita countries, most are from Europe and Asia. 
- Australia has the 11th highest GDP per Capita, 13th highest GDP and 52nd largest population (in the year 2018).
- Out of the 239 countries in the dataset, most of the top 10 highest <code>gdp_per_capita</code> countries have population sizes smaller than the 100 largest population countries, as indicated by the <code>population_ranking column</code>.
- The above statement makes the United States an exception, given it has the 9th highest <code>gdp_per_capita_ranking</code>, with the highest country GDP of all countries, and the 3rd highest population size of all countries.

### Exploring Changes in GDP and Population over recent Decades

Next let's investigate how the <code>population</code>, <code>gdp</code> and <code>gdp_per_capita</code> has changed over the last 10 and last 50 years. Six countries are selected here for a comparative analysis.

In [57]:
-- Select country name, all years' statistics available, corresponding year's population, gdp and gdp per capita
SELECT name, year, population, gdp, gdp / population AS gdp_per_capita
FROM countries AS c
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
-- Filter for specific countries
WHERE name IN('Australia', 'United States', 'United Kingdom', 'China', 'Brazil', 'Indonesia') AND year < 2018 AND year > 1940
-- Order by the country name ascending
ORDER BY name ASC;

,name,year,population,gdp,gdp_per_capita
0,Australia,1960,10276477,18577668272,1807.7857
1,Australia,1961,10483000,19653936611,1874.8389
2,Australia,1962,10742000,19892485161,1851.8419
3,Australia,1963,10950000,21507447643,1964.1505
4,Australia,1964,11167000,23764139321,2128.0684
...,...,...,...,...,...
336,United States,2013,316057727,16784800000000,53106.7541
337,United States,2014,318386421,17521700000000,55032.8118
338,United States,2015,320742673,18219300000000,56803.4800
339,United States,2016,323071342,18707200000000,57904.2384


### __Insights on changes in GDP and Population over recent Decades__

- The graph shows a drastic difference in increases to GDP per Capita over time between a first world country (US, UK and Australia) compared to a developing country (China, Brazil and Indonesia).
- The United States has had a consistent rise in its GDP per Capita since the 1970's.
- China has only seen a gradual amount of increase to its GDP per Capita from the mid 1990's onwards, despite the country currently having the 2nd highest GDP after the US. Even then, the current GDP per Capita is a long way away from US or Australia. 
- Australia has a higher GDP per Capita than the US between 2010 and 2014, where it also reached its peak GDP per capita.

## 🏡 2. Exploring Population Growth of most Populous Countries

*How much growth have the most populous countries experienced over recent decades?*

Let's begin by creating subsets of the database tables which correspond to 3 different key years: 2018 (the latest year available in the database), 2008 and 1968. Then, let's calculate percentages on the difference in total population between these years to gauge the percentage difference in population.

In [74]:
-- Set up Common Table Expression (CTE) based on year 2018
WITH pop_year_2018 AS
	(SELECT name, population AS pop_2018
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 2018),
-- Set up the year 2008 CTE
pop_year_2008 AS
	(SELECT name, population AS pop_2008
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 2008),
-- Set up the year 1968 CTE
pop_year_1968 AS
	(SELECT name, population AS pop_1968
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 1968)
-- Select country name, population in 2018, 2008 and 1968, population percentage changes over the last 10 and 50 years
SELECT 
	pop_year_2018.name AS country_name, 
	pop_2018,
    pop_2008,
    pop_1968,
    (pop_2018 - pop_2008)/pop_2018*100 AS pop_perc_change_last_10_years,
    (pop_2018 - pop_1968)/pop_2018*100 AS pop_perc_change_last_50_years
-- Join the CTE's created
FROM pop_year_2018
LEFT JOIN pop_year_2008
ON pop_year_2018.name = pop_year_2008.name
LEFT JOIN pop_year_1968
ON pop_year_2018.name = pop_year_1968.name
-- Sort by population 2018
ORDER BY pop_2018 DESC
LIMIT 20;

,country_name,pop_2018,pop_2008,pop_1968,pop_perc_change_last_10_years,pop_perc_change_last_50_years
0,China,1392730000,1324655000,774510000.0,4.8879,44.3891
1,India,1352617328,1200669765,531513824.0,11.2336,60.7048
2,United States,327167434,304093966,200706000.0,7.0525,38.6534
3,Indonesia,267663435,235469762,108779924.0,12.0277,59.3594
4,Pakistan,212215030,171648986,55102684.0,19.1155,74.0345
5,Brazil,209469333,192030362,90387079.0,8.3253,56.8495
6,Nigeria,195874740,150269623,53506196.0,23.2828,72.6835
7,Bangladesh,161356039,144304167,60918454.0,10.5679,62.2459
8,Russian Federation,144478050,142742350,NaN,1.2014,NaN
9,Japan,126529100,128063000,101061000.0,-1.2123,20.1283


### Insights on Population Growth of most Populous Countries

- Out of the 10 most populous countries, Nigeria and Pakistan have seen the highest proportional increase to their populations in their last 10 and 50 years.
- The 2 most populous countries, China and India, are the only countries to have over 1.3 billion people, with the next largest population being the US at 327 million people.
- Although Japan has the 10th largest population size, the country has seen a decline of 1.21% of its population size over the last decade.

Before moving on, let's see how the world's total population has changed over the last 10 and 50 years...

In [61]:
-- Set up Common Table Expression (CTE) based on year 2018
WITH pop_year_2018 AS
	(SELECT name, population AS pop_2018
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 2018),
-- Set up the year 2008 CTE
pop_year_2008 AS
	(SELECT name, population AS pop_2008
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 2008),
-- Set up the year 1968 CTE
pop_year_1968 AS
	(SELECT name, population AS pop_1968
	FROM countries AS c
	LEFT JOIN country_stats AS cs
	ON c.country_id = cs.country_id 
	WHERE year = 1968)
-- Select aggregated sum values for population in each year, and population percentage increase
SELECT 
	SUM(pop_2018) AS world_population_2018,
    SUM(pop_2008) AS world_population_2008,
    SUM(pop_1968) AS world_population_1968,
    (SUM(pop_2018) - SUM(pop_2008)) / SUM(pop_2018) * 100 AS world_pop_perc_change_10_years,
    (SUM(pop_2018) - SUM(pop_1968)) / SUM(pop_2018) * 100 AS world_pop_perc_change_50_years
-- Join the CTE's created
FROM pop_year_2018
LEFT JOIN pop_year_2008
ON pop_year_2018.name = pop_year_2008.name
LEFT JOIN pop_year_1968
ON pop_year_2018.name = pop_year_1968.name
-- Sort by population 2018
ORDER BY pop_2018 DESC
LIMIT 30;

,world_population_2018,world_population_2008,world_population_1968,world_pop_perc_change_10_years,world_pop_perc_change_50_years
0,7358987174,6521968829,2880206691,11.3741,60.8614


### Insights on World Population Growth

- Overall, the data indicates that the __world's population__ has increased by 11.37% over the last 10 years, and 60.86% over the last 50 years. Although the former figure of 11% appears correct, the latter shows a discrepancy to the 53% calculated based on the [Worldometer Website](https://www.worldometers.info/world-population/world-population-by-year/). This could be due to missing or incorrect data in the database of some country populations for specific years (e.g. Russia is missing its 1968 population as seen in an earlier query output).
- 53% increase to the world's population over 50 years indicates on average a 10% increase in the population each year.

## 👨‍👩‍👧‍👦 3. Exploring Population Density

*What are the most densely populated countries?*

Next let's identify which countries have the highest <code>population density</code> and how the density for some of these countries has changed over recent decades.

In [62]:
-- Select country name, population, conutry area, population density (population / area)
SELECT name, population, area AS country_area, population / area AS population_density
FROM countries AS c
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
-- Filter for Year 2018 only
WHERE year = 2018
-- Order by population density descending
ORDER BY population_density DESC;

,name,population,country_area,population_density
0,Macao,631636,18.0,35090.8889
1,Singapore,5638676,618.0,9124.0712
2,Hong Kong,7451000,1075.0,6931.1628
3,Bahrain,1569439,694.0,2261.4395
4,Maldives,515696,298.0,1730.5235
...,...,...,...,...
177,Suriname,575991,163265.0,3.5280
178,Iceland,353574,103000.0,3.4328
179,Australia,24992369,7741220.0,3.2285
180,Namibia,2448255,824292.0,2.9701


### Insights on Population Density

- Macao, a special administrative region of China, has the highest population density of any country in the world, being more than 3 times higher than the 2nd most densely populated country, Singapore.
- There are 6 countries that have between 1,000 and 10,000 people living per square kilometre.
- Bangladesh is the 7th most densely populated country, and the only country in the top 10 most densely populated countries that has extreme values for its <code>population</code> and <code>country_area</code> compared to the other countries.

## 💬 4. Exploring Country Languages

*Which languages are most commonly spoken? What countries speak the most different languages?*

Now let's explore which countries speak the most different languages, and languages that are spoken by the most different countries.

In [63]:
-- Select country name, the number of languages spoken at that country, and whether they have an official language
SELECT 
	name, 
	COUNT (DISTINCT language) AS number_of_languages, 
    official
-- Join relevant tables from database
FROM countries AS c
LEFT JOIN country_languages AS cl 
ON c.country_id = cl.country_id
LEFT JOIN languages AS l
ON cl.language_id = l.language_id
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
-- Filter for year 2018
WHERE year = 2018
-- GROUP BY country name
GROUP BY name
-- Order by number of languages spoken descending
ORDER BY number_of_languages DESC,
name ASC;

,name,number_of_languages,official
0,Canada,12,0
1,China,12,1
2,India,12,1
3,Russian Federation,12,1
4,United States,12,1
...,...,...,...
177,Grenada,1,0
178,Portugal,1,1
179,Saudi Arabia,1,1
180,Turks and Caicos Islands,1,1


In [64]:
-- Select unique languages, the total number of countries that speak that languages
SELECT
	DISTINCT language, 
    COUNT (DISTINCT name) AS number_of_countries_speaking_language
-- Join relevant tables from database
FROM countries AS c
LEFT JOIN country_languages AS cl 
ON c.country_id = cl.country_id
LEFT JOIN languages AS l
ON cl.language_id = l.language_id
LEFT JOIN country_stats AS cs
ON c.country_id = cs.country_id
-- Filter for year 2018
WHERE year = 2018
-- GROUP BY language
GROUP BY language
-- Order by number_of_countries_speaking_language descending
ORDER BY number_of_countries_speaking_language DESC
LIMIT 20;

,language,number_of_countries_speaking_language
0,English,37
1,Arabic,29
2,Spanish,24
3,German,17
4,French,17
5,Russian,16
6,Chinese,14
7,Creole English,13
8,Ful,12
9,Turkish,11


### Insights on Country Languages

- There are 13 countries that speak 10 or more different languages and/or dialects, and 7 of these also have an official language (represented by the "1" in the <code>official</code> field).

- The language that has the __highest number of countries__ that speak it is the English language, followed by Arabic, Spanish, French and Chinese. This should not be confused, however, with the number of people that can speak the language.

## 🎉 5. Exploring National Days

*Is there a most common month for a countries' national days?*

As a final exercise, we will explore if there is a particular time of the year that countries tend to have their National Days on.

In [66]:
-- Get the month from national_day field, and the corresponding count of that month
SELECT
    COUNT (DISTINCT name)
FROM countries
-- Filter for non-null values
WHERE national_day IS NOT NULL

,COUNT (DISTINCT name)
0,154


In [67]:
-- Get the month from national_day field, and the corresponding count of that month
SELECT 
	EXTRACT(MONTH FROM national_day) AS national_day_month, 
    COUNT (DISTINCT name) AS count_country_national_day_month
FROM countries
WHERE national_day IS NOT NULL
-- GROUP BY national_day_month
GROUP BY national_day_month
-- Sort by national day month count descending
ORDER BY count_country_national_day_month DESC;

,national_day_month,count_country_national_day_month
0,7,22
1,8,21
2,9,20
3,10,18
4,11,16
5,12,10
6,1,9
7,5,9
8,3,8
9,6,8


### Insights on National Days

- Out of the countries that have National Days registed in the database, most National Days for nations globally take place on the months between July and December. It is interesting how National Days are consolidated in the latter half of the year!

## Conclusion

This project has allowed for an investigative analysis into the world of statistics about the world. It has revealed some information pertaining a nations' population, GDP, area, languages, national days, countries, continents and more.

By __using existing fields to engineer new meaningful fields we have extracted insights__ into statistics about world nations, such as calculating GDP per Capita, Percentage Change in Population, and Population Density. Aggregate functions such as <code>SUM</code> and <code>COUNT</code> were used to summarize data and return significant values as part of the statistical analysis.

Some interesting findings from the analysis included the difference in GDP per Capita and its change over time between first world and developing nations, the fact that Japan's population has declined between 2008 and 2018 despite it being one of the most populous countries, how Bangladesh is among the top 10 most densely populated nations yet it has considerably higher population and land area compared to the other 10 most densely populated nations. 

> Thanks

If you made it this far in looking through my project, thank you for your time :)